In [ ]:
!pip install auto-gptq
!pip install sentencepiece
!pip install accelerate
!pip install -i https://test.pypi.org/simple/ bitsandbytes
!pip install optimum
!pip install transformers
!pip install pip install bitsandbytes

In [15]:
#  Create  a session to access OBS
from modelarts.session import Session
session = Session()

# Local project folder
local_path = '/home/ma-user/work/'

#OBS bucket
obs_path = 'obs://my-bucket/example-data'

session.obs.download_dir(
    src_obs_dir=obs_path,
    dst_local_dir=local_path
)



100%|██████████| 15/15 [00:00<00:00, 26.33it/s]


In [44]:
# connect_fun definition
from typing import Union
import sqlite3

def connect_fun(database_name: str) -> Union[sqlite3.Connection, str]:
    try:
        conn = sqlite3.connect(database_name)
        return conn
    except sqlite3.Error as e:
        return "Sorry, we couldn't connect to the database. Please make sure the database name is correct and try again."

# query_fun definition
from typing import List

def query_fun(question: str, tables_hints: List[str], database_connection: sqlite3.Connection) -> str:
    ddl_commands = [
    "CREATE",
    "VIEW",
    "ALTER",
    "DROP",
    "PROCEDURE",
    "FUNCTION",
    "GRANT",
    "REVOKE",
    'RENAME',
    'TRUNCATE'
]

    if any(command in question.upper() for command in ddl_commands):
        return "DDL commands are not allowed."

    cursor = database_connection.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    # temp_table = cursor.fetchall()
    actual_tables = [table[0] for table in cursor.fetchall()]
    print(actual_tables)
    missing_tables = [table for table in tables_hints if table not in actual_tables]

    if missing_tables:
        return f"The following tables are not part of the database: {missing_tables}. You can use tables: {actual_tables}"

    #Create prompt
    prompt = generate_sql_question_prompt(question, tables_hints, database_connection)

    #Get Sql syntax from prompt
    question = generate_sql_syntax( prompt, tokenizer, model)
    print(question)
    #Call Sql
    cursor.execute(question)
    results = cursor.fetchall()
    if results =='':
      return 'Not able to process your question. Please provide more details.'
    cursor.close()
    results_str = "Response for given query: "
    results_str += "\n".join(map(str, results))
    return results_str


In [50]:
# connect_fun definition
from typing import Union
import sqlite3

def connect_fun(database_name: str) -> Union[sqlite3.Connection, str]:
    try:
        conn = sqlite3.connect(database_name)
        return conn
    except sqlite3.Error as e:
        return "Sorry, we couldn't connect to the database. Please make sure the database name is correct and try again."

# query_fun definition
from typing import List

def query_fun(question: str, tables_hints: List[str], database_connection: sqlite3.Connection) -> str:
    ddl_commands = [
    "CREATE",
    "VIEW",
    "ALTER",
    "DROP",
    "PROCEDURE",
    "FUNCTION",
    "GRANT",
    "REVOKE",
    'RENAME',
    'TRUNCATE'
]

    if any(command in question.upper() for command in ddl_commands):
        return "DDL commands are not allowed."

    cursor = database_connection.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    actual_tables = [table[0] for table in cursor.fetchall()]
    missing_tables = [table for table in tables_hints if table not in actual_tables]

    if missing_tables:
        return f"The following tables are not part of the database: {missing_tables}. You can use tables: {actual_tables}"

    #Create prompt
    prompt = generate_sql_question_prompt(question, tables_hints, database_connection)
    print(prompt)
    #Get Sql syntax from prompt
    sql_q = generate_sql_syntax( prompt, tokenizer, model)
    print(sql_q)
    sql_extracted = remove_substring_and_trim(sql_q,prompt)
    print('sql_extracted: ',sql_extracted )
    #Call Sql
    cursor.execute(sql_extracted)
    results = cursor.fetchall()
    print('results ', results)
    if results =='':
      return 'Not able to process your question. Please provide more details.'
    cursor.close()
    results_str = "Response for given query: "
    results_str += "\n".join(map(str, results))
    return results_str


In [ ]:
print(actual_tables)

In [17]:
from typing import List

def extract_table_names_from_sql_file(tables_hints: List[str], database_connection: sqlite3.Connection):
    """
    Extract the schema for specified tables and the top 6 unique values from the first 50 rows of each table in SQLite database.
    Safely handles missing columns and continues execution.

    Parameters:
    - tables_hints (List[str]): A list of table names or hints for which to extract the schema and data.
    - database_connection (sqlite3.Connection): The SQLite3 database connection.

    Returns:
    - str: A string containing the extracted schemas and top unique values, formatted similar to SQL CREATE TABLE statements.
    """

    # Initialize an empty string to store the final schema
    extracted_schemas = ""

    # Use the provided SQLite3 database connection
    cursor = database_connection.cursor()

    # Get all table names in the SQLite database
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    all_tables = cursor.fetchall()
    all_table_names = [table[0] for table in all_tables]

    # Filter tables based on the hints provided
    filtered_table_names = [table for table in all_table_names if any(hint in table for hint in tables_hints)]

    # Loop through each table name to extract its schema and unique values
    for table_name in filtered_table_names:
        # Get the schema for the table
        cursor.execute(f"PRAGMA table_info({table_name});")
        schema_info = cursor.fetchall()
        schema_with_data = []

        # Fetch top 6 unique data for each column in SQLite3 database from first 50 rows
        for column in schema_info:
            column_name = column[1]
            column_type = column[2]
            try:
                cursor.execute(f"SELECT DISTINCT {column_name} FROM {table_name} LIMIT 50;")
                top_50_values = cursor.fetchall()
                unique_values = []#list(set(top_50_values))
                unique_values_str = ', '.join([str(x[0]) for x in unique_values[:6] if x[0] is not None])
                if unique_values_str:  # Only append if there are unique values
                    schema_with_data.append(f"{column_name} {column_type} (ex {unique_values_str})")
                else:
                    schema_with_data.append(f"{column_name} {column_type}")
            except sqlite3.OperationalError:
                # Skip the column if there is an error
                continue

        # Combine the schema and data
        schema_with_data_str = ',\n  '.join(schema_with_data)
        extracted_schemas += f'CREATE TABLE {table_name}\n({schema_with_data_str});\n\n'

    return extracted_schemas

# # Test the function with the new SQLite database connection
# conn = sqlite3.connect('/mnt/data/tallest_buildings_global.sqlite3')
# tables_hints = ["building", "location", "height"]
# extracted_schemas_test = extract_table_names_from_sql_file(tables_hints, conn)
# conn.close()

# # print(extracted_schemas_test)


In [18]:
def generate_sql_question_prompt(natural_language_question: str, tables_hints: List[str], database_connection: sqlite3.Connection):
    """
    Generate a SQL question prompt using the table schema and a natural language question.

    Parameters:
    - natural_language_question (str): The natural language question to include in the prompt.
    - tables_hints (List[str]): A list of table names or hints for which to extract the schema and data.
    - database_connection (sqlite3.Connection): The SQLite3 database connection.

    Returns:
    - str: A string containing the SQL question prompt with the table schema and the natural language question.
    """

    # Call the function to get the table schema
    table_schema = extract_table_names_from_sql_file(tables_hints, database_connection)

    # Create the SQL question prompt
    text = f"""{table_schema}
-- Using valid SQLite, answer the following questions for the tables provided above.

--  {natural_language_question }

SELECT"""

    return text

# Test the function with a sample natural language question, table hints, and SQLite database connection
# sample_question = "What are the top 5 tallest buildings in the database?"
# conn = sqlite3.connect('/mnt/data/tallest_buildings_global.sqlite3')
# sql_question_prompt = generate_sql_question_prompt(sample_question, tables_hints, conn)
# conn.close()

# print(sql_question_prompt)


In [19]:
def generate_sql_question_prompt(natural_language_question: str, tables_hints: List[str], database_connection: sqlite3.Connection):
    """
    Generate a SQL question prompt using the table schema and a natural language question.

    Parameters:
    - natural_language_question (str): The natural language question to include in the prompt.
    - tables_hints (List[str]): A list of table names or hints for which to extract the schema and data.
    - database_connection (sqlite3.Connection): The SQLite3 database connection.

    Returns:
    - str: A string containing the SQL question prompt with the table schema and the natural language question.
    """

    # Call the function to get the table schema
    table_schema = extract_table_names_from_sql_file(tables_hints, database_connection)

    # Create the SQL question prompt
    text = f"""{table_schema}
-- Using valid SQLite, answer the following questions for the tables provided above.

--  {natural_language_question }

SELECT"""

    return text

# Test the function with a sample natural language question, table hints, and SQLite database connection
# sample_question = "What are the top 5 tallest buildings in the database?"
# conn = sqlite3.connect('/mnt/data/tallest_buildings_global.sqlite3')
# sql_question_prompt = generate_sql_question_prompt(sample_question, tables_hints, conn)
# conn.close()

# print(sql_question_prompt)


In [20]:
def generate_sql_syntax(text: str, tokenizer, model):
    """
    Generate SQL syntax using a text-based machine learning model.

    Parameters:
    - text (str): The text prompt to feed into the machine learning model.
    - tokenizer: The tokenizer object compatible with the machine learning model.
    - model: The machine learning model object.

    Returns:
    - str: The generated SQL syntax.
    """


    input_ids = tokenizer(text, return_tensors="pt").input_ids

    generated_ids = model.generate(input_ids, max_length=500)
    generated_sql = (tokenizer.decode(generated_ids[0], skip_special_tokens=True))

    return generated_sql

In [3]:
# import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch
tokenizer = AutoTokenizer.from_pretrained("NumbersStation/nsql-llama-2-7B", resume_download=True)
model = AutoModelForCausalLM.from_pretrained("NumbersStation/nsql-llama-2-7B", resume_download=True, torch_dtype=torch.bfloat16)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
conn = 'example-covid-vaccinations.sqlite3'
query = "What's the statistic code used for fully vaccinated?"
tablelist = ["covid_vaccinations"]
connect_db =connect_fun(conn)
z = query_fun(query,tablelist,connect_db)
print(z)

In [ ]:

conn = 'example-covid-vaccinations.sqlite3'
query = "How many different age groups were tracked for covid vacciations?"
tablelist = ["covid_vaccinations"]
connect_db =connect_fun(conn)
z = query_fun(query,tablelist,connect_db)
print(z)

In [ ]:

conn = 'example-covid-vaccinations.sqlite3'
query = "Which electoral area has worst latest fully vaccinated rate?"
tablelist = ["covid_vaccinations"]
connect_db =connect_fun(conn)
z = query_fun(query,tablelist,connect_db)
print(z)

In [ ]:
!wget https://irchack2023-public-datasets-examples.obs.eu-west-101.myhuaweicloud.eu/test4242.zip
!unzip test4242.zip

In [51]:
def remove_substring_and_trim(main_string: str, substring_to_remove: str) -> str:
    """
    Remove a specified substring from the main string and trim any leading/trailing whitespace and newlines.

    Parameters:
    - main_string (str): The main string from which to remove the substring.
    - substring_to_remove (str): The substring to remove from the main string.

    Returns:
    - str: The main string with the substring removed, and leading/trailing whitespace and newlines trimmed.
    """
    # Remove the substring
    modified_string = main_string.replace(substring_to_remove, '')

    # Trim leading and trailing whitespaces and newlines
    modified_string = modified_string.strip()

    return 'SELECT '+ modified_string

In [52]:
import re

# Sample list of numbers with potential GUIDs and special characters
number_list = [
    "123-456-789",
    "2ae19629-3f1e-13a3-e055-000000000001",
    "42",
    "99!@#$%^&*",
    "1ab2cd34-5678-9abc-def0-123456789abc",
]

# Regular expression pattern to match GUID-like strings
guid_pattern = r'[0-9a-fA-F]{8}-([0-9a-fA-F]{4}-){3}[0-9a-fA-F]{12}'

# Function to clean the list
def clean_list(input_list):
    cleaned_list = []
    for item in input_list:
        # Check if the item is a string
        if not isinstance(item, str):
            continue

        # Remove special characters
        item = re.sub(r'[^0-9a-zA-Z]+', '', item)

        # Check if the item looks like a GUID, and skip it if it does
        if re.match(guid_pattern, item):
            continue

        cleaned_list.append(item)

    return cleaned_list



In [54]:
from typing import List

def extract_table_names_from_sql_file(tables_hints: List[str], database_connection: sqlite3.Connection):
    """
    Extract the schema for specified tables and the top 6 unique values from the first 50 rows of each table in SQLite database.
    Safely handles missing columns and continues execution.

    Parameters:
    - tables_hints (List[str]): A list of table names or hints for which to extract the schema and data.
    - database_connection (sqlite3.Connection): The SQLite3 database connection.

    Returns:
    - str: A string containing the extracted schemas and top unique values, formatted similar to SQL CREATE TABLE statements.
    """

    # Initialize an empty string to store the final schema
    extracted_schemas = ""

    # Use the provided SQLite3 database connection
    cursor = database_connection.cursor()

    # Get all table names in the SQLite database
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    all_tables = cursor.fetchall()
    all_table_names = [table[0] for table in all_tables]

    # Filter tables based on the hints provided
    filtered_table_names = [table for table in all_table_names if any(hint in table for hint in tables_hints)]

    # Loop through each table name to extract its schema and unique values
    for table_name in filtered_table_names:
        # Get the schema for the table
        cursor.execute(f"PRAGMA table_info({table_name});")
        schema_info = cursor.fetchall()
        schema_with_data = []

        # Fetch top 6 unique data for each column in SQLite3 database from first 50 rows
        for column in schema_info:
            column_name = column[1]
            column_type = column[2]
            try:
                cursor.execute(f"SELECT DISTINCT {column_name} FROM {table_name} LIMIT 50;")
                top_50_values = cursor.fetchall()
                unique_values_temp = list(set(top_50_values))
                # unique_values =clean_list(unique_values_temp)
                unique_values_str = ', '.join([str(x[0]) for x in unique_values_temp[:6] if x[0] is not None])
                print('uvs: ',unique_values_str)
                if unique_values_str:  # Only append if there are unique values
                    # schema_with_data.append(f"{column_name} {column_type} (ex {unique_values_str})")
                    schema_with_data.append(f"{column_name} {column_type}")
                else:
                    schema_with_data.append(f"{column_name} {column_type}")
            except sqlite3.OperationalError:
                # Skip the column if there is an error
                continue

        # Combine the schema and data
        schema_with_data_str = ',\n  '.join(schema_with_data)
        extracted_schemas += f'CREATE TABLE {table_name}\n({schema_with_data_str});\n\n'

    return extracted_schemas

# # Test the function with the new SQLite database connection
# conn = sqlite3.connect('/mnt/data/tallest_buildings_global.sqlite3')
# tables_hints = ["building", "location", "height"]
# extracted_schemas_test = extract_table_names_from_sql_file(tables_hints, conn)
# conn.close()

# # print(extracted_schemas_test)
def generate_sql_question_prompt(natural_language_question: str, tables_hints: List[str], database_connection: sqlite3.Connection):
    """
    Generate a SQL question prompt using the table schema and a natural language question.

    Parameters:
    - natural_language_question (str): The natural language question to include in the prompt.
    - tables_hints (List[str]): A list of table names or hints for which to extract the schema and data.
    - database_connection (sqlite3.Connection): The SQLite3 database connection.

    Returns:
    - str: A string containing the SQL question prompt with the table schema and the natural language question.
    """

    # Call the function to get the table schema
    table_schema = extract_table_names_from_sql_file(tables_hints, database_connection)

    # Create the SQL question prompt
    text = f"""{table_schema}
-- Using valid SQLite, answer the following questions for the tables provided above. Only provide resultant SQL query.

--  {natural_language_question }

SELECT"""

    return text

# Test the function with a sample natural language question, table hints, and SQLite database connection
# sample_question = "What are the top 5 tallest buildings in the database?"
# conn = sqlite3.connect('/mnt/data/tallest_buildings_global.sqlite3')
# sql_question_prompt = generate_sql_question_prompt(sample_question, tables_hints, conn)
# conn.close()

# print(sql_question_prompt)


In [56]:
def generate_sql_syntax(text: str, tokenizer, model):
    """
    Generate SQL syntax using a text-based machine learning model.

    Parameters:
    - text (str): The text prompt to feed into the machine learning model.
    - tokenizer: The tokenizer object compatible with the machine learning model.
    - model: The machine learning model object.

    Returns:
    - str: The generated SQL syntax.
    """


    input_ids = tokenizer(text, return_tensors="pt").input_ids

    generated_ids = model.generate(input_ids, max_length=500)
    generated_sql = (tokenizer.decode(generated_ids[0], skip_special_tokens=True))

    return generated_sql

In [57]:
# connect_fun definition
from typing import Union
import sqlite3

def connect_fun(database_name: str) -> Union[sqlite3.Connection, str]:
    try:
        conn = sqlite3.connect(database_name)
        return conn
    except sqlite3.Error as e:
        return "Sorry, we couldn't connect to the database. Please make sure the database name is correct and try again."

# query_fun definition
from typing import List

def query_fun(question: str, tables_hints: List[str], database_connection: sqlite3.Connection) -> str:
    ddl_commands = [
    "CREATE",
    "VIEW",
    "ALTER",
    "DROP",
    "PROCEDURE",
    "FUNCTION",
    "GRANT",
    "REVOKE",
    'RENAME',
    'TRUNCATE'
]

    if any(command in question.upper() for command in ddl_commands):
        return "DDL commands are not allowed."

    cursor = database_connection.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    actual_tables = [table[0] for table in cursor.fetchall()]
    missing_tables = [table for table in tables_hints if table not in actual_tables]

    if missing_tables:
        return f"The following tables are not part of the database: {missing_tables}. You can use tables: {actual_tables}"

    #Create prompt
    prompt = generate_sql_question_prompt(question, tables_hints, database_connection)
    print(prompt)
    #Get Sql syntax from prompt
    sql_q = generate_sql_syntax( prompt, tokenizer, model)
    print(sql_q)
    sql_extracted = remove_substring_and_trim(sql_q,prompt)
    print('sql_extracted: ',sql_extracted )
    #Call Sql
    cursor.execute(sql_extracted)
    results = cursor.fetchall()
    print('results ', results)
    if results =='':
      return 'Not able to process your question. Please provide more details.'
    cursor.close()
    results_str = "Response for given query: "
    results_str += "\n".join(map(str, results))
    return results_str


In [58]:
def check_column_name(column_name: str) -> bool:
    """
    Check if the last two characters of a column name are digits.

    Parameters:
    - column_name (str): The column name to check.

    Returns:
    - bool: True if the last two characters are not digits, False otherwise.
    """
    return not column_name[-2:].isdigit()

In [59]:
def remove_substring_and_trim(main_string: str, substring_to_remove: str) -> str:
    """
    Remove a specified substring from the main string and trim any leading/trailing whitespace and newlines.

    Parameters:
    - main_string (str): The main string from which to remove the substring.
    - substring_to_remove (str): The substring to remove from the main string.

    Returns:
    - str: The main string with the substring removed, and leading/trailing whitespace and newlines trimmed.
    """
    # Remove the substring
    modified_string = main_string.replace(substring_to_remove, '')

    # Trim leading and trailing whitespaces and newlines
    modified_string = modified_string.strip()

    return 'SELECT '+ modified_string



In [ ]:
import test4242
test4242.run_all_tests(connect_fun, query_fun)